### Design a Python Method to Obtain the Latest Constructor Standings

To get the latest standings in the season.

In [1]:
import pandas as pd
import requests

In [2]:
# Latest data url:
url = 'http://ergast.com/api/f1/current/last/constructorStandings.json'

In [3]:
# Request.
r = requests.get(url)
r.raise_for_status()
data = r.json()['MRData']['StandingsTable']['StandingsLists'][0]

In [4]:
# Store the data from the request in variables.
season_no = data['season']
round_no = data['round']
constructor_standings = data['ConstructorStandings']

In [5]:
# Manipulate the data to turn it into a more usable format.
df = pd.DataFrame(constructor_standings)
df['Constructor Name'] = df['Constructor'].apply(lambda value: value['name'])
df['Constructor URL'] = df['Constructor'].apply(lambda value: value['url'])
df['Constructor Nationality'] = df['Constructor'].apply(lambda value: value['nationality'])
df = df.set_index('position').drop(columns=['positionText', 'Constructor'])
df.index.name = 'Position'
df.columns = ['Points', 'Wins', 'Constructor Name', 'Constructor URL', 'Constructor Nationality']
df = df[['Constructor Name', 'Points', 'Wins', 'Constructor URL', 'Constructor Nationality']]
df

,Constructor Name,Points,Wins,Constructor URL,Constructor Nationality
Position,,,,,
1,Mercedes,521.5,7,http://en.wikipedia.org/wiki/Mercedes-Benz_in_...,German
2,Red Bull,510.5,10,http://en.wikipedia.org/wiki/Red_Bull_Racing,Austrian
3,Ferrari,287.5,0,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Italian
4,McLaren,256,1,http://en.wikipedia.org/wiki/McLaren,British
5,Alpine F1 Team,112,1,http://en.wikipedia.org/wiki/Alpine_F1_Team,French
6,AlphaTauri,112,0,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri,Italian
7,Aston Martin,68,0,http://en.wikipedia.org/wiki/Aston_Martin_in_F...,British
8,Williams,23,0,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,British
9,Alfa Romeo,11,0,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Swiss


In [6]:
# Create a filename.
filename = f"{season_no}_r{round_no}_Constructor_Standings.csv"

In [7]:
# Save the data to the filename.
df.to_csv(f"../api_data/transformed_data/{filename}")